In [ ]:
import pandas as pd
import numpy as np
from scipy.linalg import norm

In [4]:
import plotly_express as px
import seaborn as sns

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [6]:
sns.set()

In [9]:
config = configparser.ConfigParser()
config.read("C:/Users/chris/Documents/UVA/DS_5001/env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
local_lib = config['DEFAULT']['local_lib']

In [10]:
import sys
sys.path.append(local_lib)

In [11]:
source_files = f'{data_home}/gutenberg'
# data_prefix = 'eliot'

In [12]:
OHCO = ['book_id', 'chap_id', 'para_num', 'sent_num', 'token_num']

In [13]:
output_dir = config['DEFAULT']['output_dir']
out_path = f'{output_dir}'

In [14]:
LIB = pd.read_csv(f'{output_dir}/shelley-LIB.csv').set_index('book_id')
VOCAB = pd.read_csv(f'{output_dir}/shelley-VOCAB.csv').set_index('term_str')
CORPUS = pd.read_csv(f'{output_dir}/shelley-CORPUS.csv').set_index(OHCO)

In [15]:
# custom BOW function
def create_BOW(corpus, ohco_level):
    OHCO = ['book_id', 'chap_id', 'para_num', 'sent_num', 'token_num']
    
    bags = dict(
        SENTS = OHCO[:4],
        PARAS = OHCO[:3],
        CHAPS = OHCO[:2],
        BOOKS = OHCO[:1]
    )
    
    BOW = corpus.groupby(bags[ohco_level]+['term_str']).term_str.count().to_frame('n') 
    
    return BOW

In [16]:
def gather_docs(CORPUS, ohco_level, term_col='term_str'):
    OHCO = CORPUS.index.names
    CORPUS[term_col] = CORPUS[term_col].astype('str')
    DOC = CORPUS.groupby(OHCO[:ohco_level])[term_col].apply(lambda x:' '.join(x)).to_frame('doc_str')
    return DOC

In [22]:
def TFIDF_matrix(BOW, tf_method, CORPUS, ocholevel = 2):

    # DTCM = BOW.n.unstack(fill_value=0)
    DTCM = BOW.n.groupby(level=[0, 1, 2]).sum().unstack(fill_value=0)
    if tf_method == 'sum':
        TF = DTCM.T / DTCM.T.sum()
    elif tf_method == 'max':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'log':
        TF = np.log2(1 + DTCM.T)
    elif tf_method == 'raw':
        TF = DTCM.T
    elif tf_method == 'double_norm':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
    TF = TF.T
    DF = DTCM.astype('bool').sum() 
    N = DTCM.shape[0]
    IDF = IDF = np.log2(N / DF)
    TFIDF = TF * IDF

    DOC = gather_docs(CORPUS, 2)
    DOC['n_tokens'] = DOC.doc_str.apply(lambda x: len(x.split()))
    ngram_range = (1,2)
    n_terms = 4000
    count_engine = CountVectorizer(
        stop_words = 'english',
        ngram_range = ngram_range,
        max_features = n_terms)
    X = count_engine.fit_transform(DOC.doc_str)
    DTM = pd.DataFrame(X.toarray(), 
    columns=count_engine.get_feature_names_out(), 
    index=DOC.index)
    VOCAB = DTM.sum().to_frame('n')
    VOCAB.index.name = 'term_str'
    VOCAB['df'] = DTM.astype(bool).sum()
    VOCAB['dfidf'] = VOCAB.df * np.log2(len(DTM)/VOCAB.df)
    VOCAB['dp'] = VOCAB.df / len(DTM)
    VOCAB['di'] = np.log2(1/VOCAB.dp)
    VOCAB['dh'] = VOCAB.dp * VOCAB.di
    VOCAB['n_chars'] = VOCAB.apply(lambda x: len(x.name), 1)
    VOCAB['n_tokens'] = VOCAB.apply(lambda x: len(x.name.split()), 1)
    VOCAB.sort_index()
    
    return TFIDF, DTM

In [17]:
BOW = create_BOW(CORPUS, 'CHAPS')

In [26]:
BOW

n
book_id chap_id term_str      
134     14      appendix     1
        17      1            1
                2            2
                a           26
                abortion     1
...                         ..
41445   23      your        36
                yours        1
                yourself     3
                yourselves   1
                youth        1

[159793 rows x 1 columns]

In [23]:
TFIDF, DTM = TFIDF_matrix(BOW, 'sum', CORPUS)

In [27]:
TFIDF

term_str                1   10  100  102  103  105  10th   11  116      11th  \
book_id chap_id                                                                
134     14       0.000000  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.000000   
        17       0.005314  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.000000   
        18       0.000000  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.000000   
        19       0.000000  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.000000   
        20       0.000000  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.000000   
...                   ...  ...  ...  ...  ...  ...   ...  ...  ...       ...   
41445   19       0.000000  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.000000   
        20       0.000000  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.000000   
        21       0.000000  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.000000   
        22       0.000000  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.000000   
        23       0.000000  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.000885   

term_str         ...  zephyrs  zest  zone    à  æolian  æra  être  œdipus  \
book_id chap_id  ...                                                        
134     14       ...      0.0   0.0   0.0  0.0     0.0  0.0   0.0     0.0   
        17       ...      0.0   0.0   0.0  0.0     0.0  0.0   0.0     0.0   
        18       ...      0.0   0.0   0.0  0.0     0.0  0.0   0.0     0.0   
        19       ...      0.0   0.0   0.0  0.0     0.0  0.0   0.0     0.0   
        20       ...      0.0   0.0   0.0  0.0     0.0  0.0   0.0     0.0   
...              ...      ...   ...   ...  ...     ...  ...   ...     ...   
41445   19       ...      0.0   0.0   0.0  0.0     0.0  0.0   0.0     0.0   
        20       ...      0.0   0.0   0.0  0.0     0.0  0.0   0.0     0.0   
        21       ...      0.0   0.0   0.0  0.0     0.0  0.0   0.0     0.0   
        22       ...      0.0   0.0   0.0  0.0     0.0  0.0   0.0     0.0   
        23       ...      0.0   0.0   0.0  0.0     0.0  0.0   0.0     0.0   

term_str         œrta   ας  
book_id chap_id             
134     14        0.0  0.0  
        17        0.0  0.0  
        18        0.0  0.0  
        19        0.0  0.0  
        20        0.0  0.0  
...               ...  ...  
41445   19        0.0  0.0  
        20        0.0  0.0  
        21        0.0  0.0  
        22        0.0  0.0  
        23        0.0  0.0  

[158 rows x 20528 columns]

In [29]:
DTM

term_str         1819  abandoned  abhorrence  abilities  abject  able  abode  \
book_id chap_id                                                                
134     14          0          0           0          0       0     0      0   
        17          0          0           0          0       0     0      0   
        18          0          1           0          1       0     1      0   
        19          0          0           1          0       0     0      0   
        20          0          0           0          1       0     0      1   
...               ...        ...         ...        ...     ...   ...    ...   
41445   19          0          0           1          0       0     1      0   
        20          0          0           0          0       0     0      2   
        21          0          0           0          0       0     0      0   
        22          0          0           0          0       0     0      0   
        23          0          1           2          0       0     0      0   

term_str         abroad  absence  absent  ...  yielding  yoke  young  \
book_id chap_id                           ...                          
134     14            0        0       0  ...         0     0      0   
        17            0        0       0  ...         0     0      0   
        18            0        0       0  ...         0     0      0   
        19            0        0       1  ...         0     0      1   
        20            0        1       0  ...         0     0      1   
...                 ...      ...     ...  ...       ...   ...    ...   
41445   19            0        0       0  ...         0     0      0   
        20            0        0       0  ...         0     0      2   
        21            0        1       0  ...         0     0      1   
        22            1        0       0  ...         0     0      0   
        23            0        0       0  ...         0     0      0   

term_str         young man  young people  young woman  younger  youth  \
book_id chap_id                                                         
134     14               0             0            0        0      0   
        17               0             0            0        0      0   
        18               0             0            0        0      0   
        19               0             0            0        0      1   
        20               1             0            0        0      3   
...                    ...           ...          ...      ...    ...   
41445   19               0             0            0        0      0   
        20               2             0            0        0      0   
        21               0             0            0        0      0   
        22               0             0            0        0      1   
        23               0             0            0        1      1   

term_str         youthful  zeal  
book_id chap_id                  
134     14              0     0  
        17              0     0  
        18              0     0  
        19              0     0  
        20              0     0  
...                   ...   ...  
41445   19              0     0  
        20              1     0  
        21              0     0  
        22              0     0  
        23              0     0  

[158 rows x 4000 columns]

In [31]:

VOCAB_REDUCED = VOCAB[VOCAB.max_pos.isin(['NN', 'NNS']) & ~VOCAB.max_pos.isin(['NNP'])].sort_values('dfidf', ascending=False).head(1000)
TFIDF_REDUCED = TFIDF[VOCAB_REDUCED.index]
TFIDF_REDUCED

term_str         conversation  passions     point    months      word  \
book_id chap_id                                                         
134     14           0.000000  0.000000  0.000000  0.000000  0.000000   
        17           0.000000  0.001628  0.000000  0.001628  0.000000   
        18           0.000663  0.000663  0.000663  0.000663  0.000000   
        19           0.000865  0.000865  0.000000  0.000000  0.000000   
        20           0.001122  0.001682  0.000000  0.000000  0.000000   
...                       ...       ...       ...       ...       ...   
41445   19           0.000000  0.000418  0.000418  0.000835  0.000000   
        20           0.000000  0.000000  0.000393  0.001571  0.000000   
        21           0.000872  0.000436  0.000000  0.000436  0.000436   
        22           0.000000  0.000000  0.000000  0.001147  0.000000   
        23           0.000175  0.000526  0.000350  0.000350  0.000000   

term_str             door  understanding     peace  solitude  order  ...  \
book_id chap_id                                                      ...   
134     14       0.000000       0.000000  0.000000  0.000000    0.0  ...   
        17       0.000000       0.000000  0.000000  0.000000    0.0  ...   
        18       0.000000       0.001325  0.000000  0.000651    0.0  ...   
        19       0.000865       0.000000  0.000000  0.000000    0.0  ...   
        20       0.000000       0.001122  0.000000  0.000000    0.0  ...   
...                   ...            ...       ...       ...    ...  ...   
41445   19       0.002088       0.000000  0.000821  0.000000    0.0  ...   
        20       0.000786       0.000393  0.000000  0.000000    0.0  ...   
        21       0.000000       0.000000  0.002572  0.000429    0.0  ...   
        22       0.000000       0.000000  0.001128  0.000000    0.0  ...   
        23       0.000175       0.000000  0.000689  0.000344    0.0  ...   

term_str         gates    faith  trade   sailors  falsehood  bounds     brain  \
book_id chap_id                                                                 
134     14         0.0  0.00000    0.0  0.000000   0.000000     0.0  0.000000   
        17         0.0  0.00000    0.0  0.000000   0.000000     0.0  0.000000   
        18         0.0  0.00000    0.0  0.000000   0.000000     0.0  0.001474   
        19         0.0  0.00000    0.0  0.000000   0.000000     0.0  0.000000   
        20         0.0  0.00000    0.0  0.000000   0.000000     0.0  0.000000   
...                ...      ...    ...       ...        ...     ...       ...   
41445   19         0.0  0.00000    0.0  0.000000   0.000000     0.0  0.000000   
        20         0.0  0.00000    0.0  0.000000   0.000000     0.0  0.000000   
        21         0.0  0.00000    0.0  0.000000   0.000000     0.0  0.000000   
        22         0.0  0.00000    0.0  0.000000   0.001276     0.0  0.000000   
        23         0.0  0.00039    0.0  0.001559   0.000000     0.0  0.000000   

term_str         thousands       ice    plains  
book_id chap_id                                 
134     14             0.0  0.000000  0.000000  
        17             0.0  0.000000  0.000000  
        18             0.0  0.000000  0.000000  
        19             0.0  0.000000  0.000000  
        20             0.0  0.000000  0.001248  
...                    ...       ...       ...  
41445   19             0.0  0.000000  0.000000  
        20             0.0  0.000000  0.000000  
        21             0.0  0.000970  0.000000  
        22             0.0  0.001276  0.000000  
        23             0.0  0.007016  0.000000  

[158 rows x 1000 columns]

In [32]:
BOW.to_csv(f'{output_dir}/shelley-BOW.csv')
TFIDF.to_csv(f'{output_dir}/shelley-TFIDF.csv')
DTM.to_csv(f'{output_dir}/shelley-DTM.csv')
VOCAB_REDUCED.to_csv(f'{output_dir}/shelley-VOCAB-REDUCED.csv')
TFIDF_REDUCED.to_csv(f'{output_dir}/shelley-TFIDF-REDUCED.csv')